In [66]:
import gensim
import keras
from keras import utils
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
import multiprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
import numpy as np
import os
from operator import getitem
import pandas as pd
import re
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [47]:
stopwords_set = set(stopwords.words('english'))
required = {"needn", "doesn", "won", "shan't", "don", "ain", "not", "didn", "hadn", "haven't", "couldn't", "wasn't", "aren't", "isn", "needn't", "aren", "wouldn", "shouldn", "hasn't", "shan", "no", "wasn", "nor", "hasn", "mightn", "doesn't", "against", "wouldn't", "couldn", "hadn't", "isn't", "mustn", "don't", "weren't", "haven", "mustn't", "shouldn't", "weren", "won't", "mightn't"}
stopwords_set -= required
print(stopwords_set)
dictionary = set(nltk.corpus.words.words())

{'up', 'down', 'these', 'and', 'about', 'him', 'we', 'himself', 'to', 'if', 'where', 'only', "didn't", 'how', "she's", 'whom', 'an', 'ourselves', 'myself', 'but', 'your', "should've", 'what', 'few', 'do', 'have', 'each', 'themselves', 'or', 'm', "you'd", 'on', 'because', 'there', 'will', 'me', 'it', 'yourselves', 's', 'too', 'before', 'own', 'from', 'should', 'my', 'can', 'am', 'doing', 'is', 'further', 'until', 'which', 'into', 'again', 'such', 'ma', 'you', 'yours', 'he', 'does', 'very', 'just', 'out', 'a', 'she', 'his', 'who', 'while', 'are', 'over', 're', 'they', 'other', 'at', 'same', 'did', 'some', 'having', 'of', 'more', 'then', 'theirs', 'y', 'most', 'as', 'during', 'here', 'in', 've', 'herself', 'now', 'this', 'being', 'yourself', 'has', 'hers', 'for', 'll', 'any', 'i', 'itself', "you'll", 't', "it's", 'their', 'were', 'with', 'that', "you're", 'through', "you've", 'after', 'so', "that'll", 'o', 'why', 'than', 'off', 'ours', 'both', 'once', 'them', 'd', 'been', 'our', 'all', 't

In [48]:
df = pd.read_csv("FinalTweetList_train.csv", header = None)

In [49]:
#upsample compliments to equalise classes
'''
compliment = df[df[1]=='compliment']
displeasure = df[df[1]=='displeasure']
miscellaneous = df[df[1]=='miscellaneous']
size = min(len(displeasure), len(miscellaneous))
print(size)
compliment_extra = pd.DataFrame()
while(len(compliment) + len(compliment_extra) < size):
    compliment_extra = compliment_extra.append(compliment.sample(min(size - (len(compliment) + len(compliment_extra)),len(compliment))))
compliment = compliment_extra.append(compliment)

df = compliment.append([displeasure, miscellaneous])
df = df.reset_index(drop = True)
print(df)
'''
# partially stratified upsample


"\ncompliment = df[df[1]=='compliment']\ndispleasure = df[df[1]=='displeasure']\nmiscellaneous = df[df[1]=='miscellaneous']\nsize = min(len(displeasure), len(miscellaneous))\nprint(size)\ncompliment_extra = pd.DataFrame()\nwhile(len(compliment) + len(compliment_extra) < size):\n    compliment_extra = compliment_extra.append(compliment.sample(min(size - (len(compliment) + len(compliment_extra)),len(compliment))))\ncompliment = compliment_extra.append(compliment)\n\ndf = compliment.append([displeasure, miscellaneous])\ndf = df.reset_index(drop = True)\nprint(df)\n"

In [50]:
def processTweet(tweet):
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    #Convert to lower case
    tweet = tweet.lower()
    return tweet

for i in range(len(df[0])):
    df[0][i] = processTweet(df[0][i])

print(df)

                                                       0              1
0      at_user at_user at_user at_user at_user at_use...     compliment
1      rt at_user at_user at_user at_user at_user at_...  miscellaneous
2      first hand experience of the cashrush process ...     compliment
3      at_user at_user hi, sorry for the trouble. we'...  miscellaneous
4               at_user at_user the link is not opening.  miscellaneous
...                                                  ...            ...
11374  at_user in queue since 7 am at_user out of cur...  miscellaneous
11375  at_user at_user at_user at_user at_user at_use...  miscellaneous
11376  thank you bankers for your kind co operation. ...     compliment
11377  hell lot of sms by at_user at_user but no atms...     compliment
11378  at_user at_user at_user at_user atps not worki...    displeasure

[11379 rows x 2 columns]


In [51]:
#Stopword removal, non-alphabet tokens removed, optional non-english word removal

#train, test = train_test_split(df, test_size = (500 if len(feats)>500 else 0.25), stratify = df[1], random_state = 21)
#train, test = train_test_split(df, test_size = (500 if len(feats)>500 else 0.25), random_state = 21)
#train = train.reset_index(drop = True)
#test = test.reset_index(drop = True)
train = df

def word_clean(words):
    #return ' '.join([word for word in words if (word.isalpha())])
    #return ' '.join([word for word in words if (word.isalpha() and word in dictionary)])
    return ' '.join([word for word in words if (word.isalpha() and word not in stopwords_set)])
    #return ' '.join([word for word in words if (word.isalpha() and word not in stopwords_set and word in dictionary)])
    
def clean_tweets(dataframe):
    return [word_clean(WordPunctTokenizer().tokenize(dataframe[0][i])) for i in range(len(dataframe[0]))]

x_train = clean_tweets(train)
y_train = train[1]
print(len(y_train))

11379


In [52]:
x_train = [gensim.utils.simple_preprocess(text) for text in x_train]
#Continous bag of words used
w2v_model = gensim.models.Word2Vec(min_count=1, window=5,
                                        size=300,
                                        workers=multiprocessing.cpu_count())
w2v_model.build_vocab(x_train)
w2v_model.train(x_train, total_examples=w2v_model.corpus_count, epochs=100)
w2v_words = list(w2v_model.wv.vocab)
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

print("LabelEncoder")
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
num_classes = len(label_encoder.classes_)
y_train = utils.to_categorical(y_train, num_classes)

print("Fit Tokenizer")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
x_train = keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(x_train),
                                                     maxlen=500)
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

print("Create Embedding matrix")
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))
for word, idx in word_index.items():
    if word in w2v_words:
        embedding_vector = w2v_model.wv.get_vector(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = w2v_model.wv[word]
print("Embedding matrix: %s" % str(embedding_matrix.shape))

print("Build Keras model")
print('x_train shape: %s' % str(x_train.shape))
print('y_train shape: %s' % str(y_train.shape))

k_model = Sequential()
k_model.add(Embedding(vocab_size,
                           300,
                           weights=[embedding_matrix],
                           input_length=500,
                           trainable=False))
k_model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.2))
for hidden_layer in (128, 64, 32):
    k_model.add(Dense(hidden_layer, activation='relu'))
    k_model.add(Dropout(0.2))
if num_classes > 2:
    k_model.add(Dense(num_classes, activation='softmax'))
else:
    k_model.add(Dense(num_classes, activation='sigmoid'))

k_model.compile(loss='categorical_crossentropy' if num_classes > 2 else 'binary_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])
print(k_model.summary())

# Callbacks
early_stopping = EarlyStopping(monitor='accuracy', patience=6, verbose=0, mode='max')
rop = ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='max')
callbacks = [early_stopping, rop]

print("Fit Keras model")
k_model.fit(x_train, y_train,
                 batch_size=128,
                 epochs=32,
                 callbacks=callbacks,
                 verbose=1)

print("Done")

Vocabulary size: 6985
Word2Vec trained
LabelEncoder
Fit Tokenizer
Number of unique words: 6986
Create Embedding matrix
Embedding matrix: (6986, 300)
Build Keras model
x_train shape: (11379, 500)
y_train shape: (11379, 3)
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 300)          2095800   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_______________________________

C:\Users\User\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/32
11379/11379 [==============================] - 300s 26ms/step - loss: 0.9053 - accuracy: 0.5279
Epoch 2/32
11379/11379 [==============================] - 289s 25ms/step - loss: 0.7343 - accuracy: 0.6740
Epoch 3/32
11379/11379 [==============================] - 294s 26ms/step - loss: 0.6487 - accuracy: 0.7247
Epoch 4/32
11379/11379 [==============================] - 298s 26ms/step - loss: 0.5881 - accuracy: 0.7507
Epoch 5/32
11379/11379 [==============================] - 306s 27ms/step - loss: 0.5361 - accuracy: 0.7809
Epoch 6/32
11379/11379 [==============================] - 307s 27ms/step - loss: 0.4837 - accuracy: 0.8047
Epoch 7/32
11379/11379 [==============================] - 305s 27ms/step - loss: 0.4449 - accuracy: 0.8226
Epoch 8/32
11379/11379 [==============================] - 308s 27ms/step - loss: 0.4055 - accuracy: 0.8414
Epoch 9/32
11379/11379 [==============================] - 307s 27ms/step - loss: 0.3684 - accuracy: 0.8591
Epoch 10/32
11379/11379 [============

In [67]:
# Prepare test

values = {
    "compliment": 1,
    "miscellaneous": 0.5,
    "displeasure": 0
}

results = {}

direc = os.fsencode('./test')
for File in os.listdir(direc):
    filename = os.fsdecode(File)
    try:
        test = pd.read_csv("./test/" + filename, header = None)
        for i in range(len(test[0])):
            test[0][i] = processTweet(test[0][i])
        x_test = clean_tweets(test)
        y_test = test[1]

        x_test = [gensim.utils.simple_preprocess(text) for text in x_test]
        x_test = keras.preprocessing.sequence.pad_sequences(
            tokenizer.texts_to_sequences(x_test),
            maxlen=500)

        # Predict
        confidences = k_model.predict(x_test, verbose=0)

        y_pred_1d = []

        for confidence in confidences:
            idx = np.argmax(confidence)
            y_pred_1d.append(label_encoder.classes_[idx])
            
        sum = 0
        for i in y_pred_1d:
            sum += values[i]
        result = {}
        result['performance_index'] = sum * 100 / len(y_pred_1d)
        result['classification_report'] = classification_report(y_test, y_pred_1d)
        result['tweets'] = len(y_pred_1d)
        results[filename[:-4]] = result
    except:
        pass
    
for i in sorted(results.items(), key = lambda x: ( -(getitem(x[1],'tweets')), -(getitem(x[1],'performance_index')) )):
    print((i[0] + ": ").rjust(40) + ("{:.2f}".format(i[1]['performance_index'])).rjust(6) + "% in " + str(i[1]['tweets']).rjust(4) + " tweets")
    print(i[1]['classification_report'])
    
#The toppers are:
    #1. Kotak Bank: 38.70%
    #2. Axis Bank: 34.80%
    #3. HDFC Bank: 32.90%
    #4. State Bank of India: 29.20%
    #5. ICICI Bank: 28.90%
#Absolutely correct order

                            Kotak Bank:  38.70% in  500 tweets
               precision    recall  f1-score   support

   compliment       0.90      0.92      0.91       113
  displeasure       0.91      0.90      0.91       232
miscellaneous       0.86      0.86      0.86       155

     accuracy                           0.89       500
    macro avg       0.89      0.90      0.89       500
 weighted avg       0.89      0.89      0.89       500

                             Axis Bank:  34.80% in  500 tweets
               precision    recall  f1-score   support

   compliment       0.87      1.00      0.93        41
  displeasure       0.99      0.99      0.99       201
miscellaneous       0.99      0.97      0.98       258

     accuracy                           0.98       500
    macro avg       0.95      0.99      0.97       500
 weighted avg       0.98      0.98      0.98       500

                             HDFC Bank:  32.90% in  500 tweets
               precision    recall 